# Building Multi-Agent Investment Firms with Strands SDK


## Task 3: Final Investment Recommendations

### Objective
Develop proficiency with the Strands Agent API by creating specialized agents for comprehensive market analysis and generating final trading decisions based on multi-agent research collaboration.

### Requirements
- **Target Security**: AMZN (Amazon.com Inc.)
- **Analysis Date**: 2025-08-19
- **Agent Types**: Trader Decision Agent
- **Data Sources**: Historical stock data, market indicators, news sentiment
- **Output Format**: CSV files for quantitative data, text files for qualitative analysis

### Prerequisites
- Task 1 completed: Market data collection and technical analysis
- Task 2 completed: Multi-agent research and investment plan generation

### Success Criteria
✅ Successful generation of final trading report  
✅ Successful replace Bedrock model from Nova pro to Nova lite   
✅ Clear BUY/SELL/HOLD recommendation with detailed rationale  
✅ Risk management parameters and execution strategy  
✅ Complete Task 3 challenge requirements

---

## 1. System Configuration and Initialization

### 1.1 Core Library Imports

This section imports essential libraries for system operation, including:
- **Configuration Management**: Default settings and environment variables
- **Logging Framework**: System monitoring and debugging capabilities
- **Path Management**: File system operations and directory handling

These imports establish the foundation for our multi-agent trading system.

In [ ]:
# Core system imports for multi-agent framework
import sys
import os
from pathlib import Path
from default_config import DEFAULT_CONFIG
import logging
from dotenv import load_dotenv

# Configure logging for system monitoring and debugging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Load environment variables from .env file if present
load_dotenv()

print("✅ Core libraries imported successfully")
print("✅ Logging configured for system monitoring")
print("✅ Environment variables loaded")

### 1.2 Load Configuration Variables
This section retrieves essential configuration variables from the Jupyter notebook's variable store. These variables include the Amazon Nova Pro model ID, company of interest for analysis, trade date, working directory, and other configuration parameters that were set up in previous challenges. This ensures consistency across the multi-agent investment analysis workflow.

In [ ]:
%store -r NOVA_PRO_MODEL_ID
%store -r company_of_interest
%store -r trade_date
%store -r working_dir
%store -r prefix
%store -r config

NOVA_PRO_MODEL_ID = NOVA_PRO_MODEL_ID
company_of_interest = company_of_interest
trade_date = trade_date
working_dir = working_dir
prefix = prefix
config = config

print("📊 ANALYSIS TARGET CONFIGURATION")
print("=" * 40)
print(f"Target Security   : {company_of_interest}")
print(f"Company Name      : Amazon.com Inc.")
print(f"Exchange          : NASDAQ")
print(f"Sector            : Technology")
print(f"Analysis Date     : {trade_date}")
print("=" * 40)
print("✅ Target security configured for analysis")
# Display directory structure
print("\n📁 OUTPUT DIRECTORY STRUCTURE")
print("=" * 40)
print(f"Base Directory    : {working_dir}")
print(f"Analysis Prefix   : {prefix}")
print("=" * 40)
print("✅ Professional directory structure configured")

### 1.3 Bedrock Model Factory

This factory function creates optimally configured Bedrock model instances for financial analysis tasks. The configuration includes:

**Performance Optimizations:**
- **Extended Timeouts**: 30-minute read timeout accommodates complex analytical processes
- **Adaptive Retry Logic**: Ensures reliability in production environments with intelligent backoff
- **Connection Management**: 15-minute connection timeout prevents hanging requests

**Advanced Features:**
- **Thinking Mode**: Provides transparency in AI reasoning process (optional)
- **Token Budget Management**: Dedicated tokens for reasoning transparency
- **Temperature Control**: Balanced for financial analysis accuracy (0.7 default)

**Financial Analysis Optimizations:**
- **Extended Token Limits**: 10,000 tokens accommodate comprehensive financial reports
- **Precision Balance**: Temperature setting balances analytical precision with adaptive reasoning
- **Reliability Features**: Multiple retry attempts with exponential backoff

In [ ]:
import boto3
from botocore.config import Config
from strands.models import BedrockModel

# AWS Boto3 client configuration optimized for financial analysis
boto_client_config = Config(
    read_timeout=1800,      # 30 minutes - accommodates complex analyses
    connect_timeout=900,    # 15 minutes - ensures reliable connections
    retries=dict(max_attempts=3, mode="adaptive"),  # Adaptive retry strategy
)

def get_model(model_id=NOVA_PRO_MODEL_ID, thinking=False,
              temperature=0.7, max_tokens=10000):
    """
    Create and return a Bedrock model instance optimized for financial analysis.

    Parameters:
    -----------
    model_id : str
        The specific AWS Bedrock model identifier to use
    thinking : bool
        Enable thinking mode for supported models (provides reasoning transparency)
    temperature : float
        Sampling temperature (0.0-1.0) - controls randomness in responses
        - 0.7 provides good balance between creativity and consistency
    max_tokens : int
        Maximum tokens in the response (affects response length)

    Returns:
    --------
    BedrockModel
        Configured model instance ready for agent integration
        
    Notes:
    ------
    - Temperature 0.7 balances analytical precision with adaptive reasoning
    - Thinking mode provides insight into the model's reasoning process
    - Extended token limit accommodates comprehensive financial reports
    """
    session = boto3.Session()

    # Configure thinking mode for enhanced reasoning transparency
    additional_request_fields = {}
    if thinking:
        additional_request_fields = {
            "thinking": {
                "type": "enabled",
                "budget_tokens": 4096,  # Dedicated tokens for reasoning process
            }
        }

    return BedrockModel(
        model_id=model_id,
        boto_session=session,
        max_tokens=max_tokens,
        temperature=temperature,
        boto_client_config=boto_client_config,
        additional_request_fields=additional_request_fields,
    )

print("✅ Bedrock model factory configured")
print("🔧 Optimized for financial analysis workloads")
print("⚙️ Extended timeouts and adaptive retry logic enabled")

### 1.4 Primary Model Initialization

Initialize the main language model that will serve as the cognitive engine for our financial analysis agents.

**Configuration Details:**
- **Model**: Amazon Nova Pro v1.0 - Latest generation reasoning model
- **Temperature**: 0.7 - Balanced creativity and consistency for financial analysis
- **Max Tokens**: 10,000 - Sufficient for comprehensive trading reports
- **Thinking Mode**: Disabled for production efficiency (can be enabled for debugging)

In [ ]:
# Initialize the primary language model for agent operations
llm = get_model(model_id=NOVA_PRO_MODEL_ID, thinking=False)

# Verify successful initialization
print(f"🧠 Language Model Initialized: {NOVA_PRO_MODEL_ID}")
print(f"🎛️ Configuration: Temperature=0.7, Max Tokens=10,000")
print(f"✅ Model ready for agent integration")

## 2. Trader Decision Agent Architecture

### 2.1 Agent Design Philosophy

The Trader Agent represents the final decision-making component in our multi-agent investment system. This agent is designed to:

**Core Responsibilities:**
- Synthesize research from multiple specialized agents
- Make definitive trading decisions (BUY/SELL/HOLD)
- Implement comprehensive risk management protocols
- Generate executable trading strategies with specific parameters

**Decision Framework:**
- **Technical Analysis Integration**: Incorporates moving averages, momentum indicators, and volume analysis
- **Fundamental Research**: Considers earnings, growth prospects, and competitive positioning
- **Market Sentiment**: Evaluates news sentiment, analyst opinions, and market psychology
- **Risk Assessment**: Implements position sizing, stop-losses, and scenario planning

**Professional Standards:**
- Follows institutional trading best practices
- Maintains detailed decision audit trails
- Implements systematic risk management
- Provides clear, actionable recommendations

In [ ]:
"""
Professional Trader Agent Implementation

This module defines the Trader agent responsible for making final trading decisions
based on comprehensive research team recommendations and market analysis. The agent 
translates investment strategies into specific, executable trading actions with 
detailed risk management parameters.

Key Features:
- Multi-factor decision analysis
- Systematic risk management
- Professional execution strategies
- Performance tracking and learning
"""

from strands import Agent


def create_trader(llm, config):
    """
    Create a Professional Trader agent that makes final trading decisions and executes strategies.
    
    This agent implements institutional-grade trading decision processes, incorporating:
    - Multi-dimensional market analysis
    - Systematic risk management protocols
    - Professional execution strategies
    - Continuous learning from past decisions
    
    Args:
        llm: The language model to use for the agent
        memory (str): Memory identifier for storing/retrieving past trading decisions
        config (dict): Configuration settings for the agent
        
    Returns:
        Agent: Configured Professional Trader agent
    """
    
    system_message = (
        """You are a Professional Trader responsible for making final trading decisions 
        and executing investment strategies. Your role is to translate research recommendations 
        into specific, actionable trading plans with clear risk management parameters.

        **Key Responsibilities:**
        1. Analyze research team recommendations and market data
        2. Make definitive trading decisions (BUY, SELL, or HOLD)
        3. Develop specific execution strategies with entry/exit criteria
        4. Implement risk management and position sizing
        5. Learn from past trading decisions and market outcomes

        **Trading Decision Framework:**

        **Market Analysis Integration:**
        - Incorporate technical analysis with fundamental research
        - Assess market timing and liquidity conditions
        - Evaluate market sentiment and momentum factors
        - Consider correlation with broader market movements
        - Analyze volume patterns and institutional activity

        **Risk Management Protocol:**
        - Define position sizing based on portfolio risk limits
        - Set stop-loss levels and profit-taking targets
        - Calculate risk-reward ratios for each trade
        - Consider portfolio correlation and diversification
        - Plan for different market scenarios and volatility

        **Execution Strategy:**
        - Determine optimal entry and exit timing
        - Choose appropriate order types and execution methods
        - Consider market impact and slippage factors
        - Plan for partial fills and position scaling
        - Monitor execution quality and market conditions

        **Decision Criteria:**
        - **BUY**: Strong conviction with favorable risk-reward and clear catalysts
        - **SELL**: Risk concerns outweigh potential returns or better opportunities exist
        - **HOLD**: Maintain current position when conditions don't warrant change
        - Always provide specific rationale for each decision

        **Performance Tracking:**
        - Monitor trade performance against expectations
        - Track risk-adjusted returns and Sharpe ratios
        - Analyze win/loss ratios and average trade outcomes
        - Identify patterns in successful and unsuccessful trades
        - Adjust trading strategies based on performance data

        **Learning and Adaptation:**
        - Review past trading decisions and their outcomes
        - Identify behavioral biases and systematic errors
        - Adapt to changing market conditions and regimes
        - Refine risk management based on historical drawdowns
        - Update trading models based on market evolution

        **Communication Requirements:**
        1. Provide clear, specific trading recommendations
        2. Include detailed rationale for each decision
        3. Specify entry/exit criteria and risk parameters
        4. Address potential scenarios and contingency plans
        5. Use professional trading terminology appropriately
        6. Always conclude with a definitive decision statement

        **Decision Output Format:**
        Your analysis should include:
        - Market assessment and key factors
        - Risk-reward analysis and position sizing
        - Entry/exit strategy and timing considerations
        - Risk management parameters (stop-loss, targets)
        - Scenario analysis and contingency planning
        - **FINAL TRANSACTION PROPOSAL: BUY/HOLD/SELL** (mandatory conclusion)

        **Critical Guidelines:**
        - Make decisive recommendations based on thorough analysis
        - Always conclude with "FINAL TRANSACTION PROPOSAL: **BUY/HOLD/SELL**"
        - Consider both technical and fundamental factors
        - Implement proper risk management for every trade
        - Learn from past mistakes to improve future performance
        - Maintain discipline and avoid emotional decision-making

        Remember: Your role is to translate research insights into profitable, 
        risk-managed trading decisions that optimize portfolio performance.
        """
    )

    # Create and configure the agent
    agent = Agent(
        model=llm,
        name="Professional_Trader",
        system_prompt=system_message,
        load_tools_from_directory=False,
    )
    
    # Set agent state for memory and configuration access
    agent.state.set("config", config)
    
    return agent

### 2.2 File Management Utilities

These utility functions provide robust file management capabilities for our trading system:

**Key Features:**
- **Structured Directory Management**: Organized file storage with consistent naming
- **UTF-8 Encoding**: Proper handling of international characters and symbols
- **Error Handling**: Graceful handling of missing files and directory creation
- **Professional Organization**: Systematic file structure for audit trails

In [ ]:
def save_as_file(text, working_dir, prefix='', file_name=''):
    """
    Save text content to a file in the specified directory structure.
    
    This function creates a professional file management system with:
    - Automatic directory creation
    - UTF-8 encoding for international compatibility
    - Structured naming conventions
    - Error handling for file operations
    
    Args:
        text (str): Content to save
        working_dir (str): Base working directory
        prefix (str): Subdirectory prefix (usually ticker_date)
        file_name (str): Name of the file to save
        
    Example:
        save_as_file("Trading decision content", "./results", "AMZN_2025-08-19", "trader_decision.txt")
    """
    # Create directory if it doesn't exist
    full_dir = os.path.join(working_dir, prefix)
    if not os.path.exists(full_dir):
        os.makedirs(full_dir, exist_ok=True)
    
    # Write content to file with UTF-8 encoding
    file_path = os.path.join(full_dir, file_name)
    with open(file_path, "w", encoding='utf-8') as f:
        f.write(text)


def read_file(working_dir, prefix='', file_name=''):
    """
    Read text content from a file in the specified directory structure.
    
    This function provides robust file reading with:
    - UTF-8 encoding support
    - Clear error messages for missing files
    - Consistent path handling
    
    Args:
        working_dir (str): Base working directory
        prefix (str): Subdirectory prefix (usually ticker_date)
        file_name (str): Name of the file to read
        
    Returns:
        str: File content
        
    Raises:
        ValueError: If the file doesn't exist with clear path information
        
    Example:
        content = read_file("./results", "AMZN_2025-08-19", "investment_plan.txt")
    """
    file_path = os.path.join(working_dir, prefix, file_name)
    if os.path.exists(file_path):
        with open(file_path, "r", encoding='utf-8') as f:
            return f.read()
    else:
        raise ValueError(f"File not found: {file_path}")

## 3. Replace Bedrock Model Nova Pro with Note Lite

This section is part of the Task 3, where you will use Amazon Bedrock to access a variety of LLMs. Amazon Nova understanding models include the following,

+ Amazon Nova Micro is a text-only model that delivers the lowest latency responses at very low cost.

+ Amazon Nova Lite is a very low cost multimodal model that is lightning fast for processing image, video, and text inputs.

+ Amazon Nova Pro is a highly capable multimodal model and our best combination of accuracy, speed, and cost for a wide range of tasks.

+ Amazon Nova Premier is our most capable multimodal model for complex tasks and our best teacher for distilling custom models for cost-effective applications.

This session you need to replace `Nova Pro` to `Note Lite` via strands agents SDK.


### 3.1 View current model configuration
You can see now you are using `amazon.nova-pro-v1:0`, Amazon Nova Lite also provides great performance for accuracy, speed, and cost.
You need to read the following documents to complete task 3. 

https://docs.aws.amazon.com/nova/latest/userguide/what-is-nova.html 

https://strandsagents.com/latest/documentation/docs/api-reference/models/#strands.models.bedrock.BedrockModel

In [ ]:
llm_config=llm.get_config()
print(llm_config)

### 3.2 Write your code here



In [ ]:
# Repace Bedrock Model Nova Pro with Note Lite
###



### 3.3 Paste output to Task 3

In [ ]:
print(llm_config)

## 4. Trading Decision Execution

### 4.1 Agent Initialization and Data Loading

This section initializes the Professional Trader agent and loads the investment plan generated from previous analysis tasks.

**Process Overview:**
1. **Agent Creation**: Initialize the Professional Trader with optimized configuration
2. **Memory Integration**: Connect to trading memory for historical decision learning
3. **Data Loading**: Import the comprehensive investment plan from Task 2
4. **Validation**: Ensure all required inputs are available for decision-making

**Input Requirements:**
- Investment plan from multi-agent research (Task 2 output)
- Historical trading memory for pattern recognition
- Market configuration and risk parameters

In [ ]:
# Initialize the Professional Trader agent
trader = create_trader(llm, config)

# Load the investment plan generated from Task 2
investment_plan = read_file(working_dir, prefix, "investment_plan.txt")

### 4.2 Final Trading Decision Generation

Execute the final trading decision process using the Professional Trader agent.

**Decision Process:**
1. **Input Analysis**: Comprehensive review of the multi-agent investment plan
2. **Memory Integration**: Leverage historical trading decisions and lessons learned
3. **Risk Assessment**: Evaluate risk-reward ratios and position sizing
4. **Market Timing**: Consider current market conditions and execution timing
5. **Final Decision**: Generate definitive BUY/SELL/HOLD recommendation

**Output Components:**
- **Market Assessment**: Current conditions and key driving factors
- **Risk Analysis**: Position sizing, stop-losses, and profit targets
- **Execution Strategy**: Entry/exit timing and order management
- **Scenario Planning**: Contingency plans for different market outcomes
- **Final Recommendation**: Clear BUY/SELL/HOLD decision with rationale

**Professional Standards:**
- Institutional-grade decision framework
- Comprehensive risk management protocols
- Clear audit trail and decision documentation
- Actionable implementation guidelines

In [ ]:
# Execute the final trading decision process
trader_decision = trader(
    f"Based on the following comprehensive investment plan for {company_of_interest} "
    f"for the trade date {trade_date}, what is your final trade decision?\n\n"
    f"Please provide a detailed analysis including market assessment, risk management, "
    f"execution strategy, and conclude with a definitive BUY/SELL/HOLD recommendation.\n\n"
    f"Investment Plan:\n{investment_plan}"
)

# Save the trading decision to file for audit trail
save_as_file(str(trader_decision), working_dir, prefix, "trader_decision.txt")

## 5. Task 3 Completion Summary

### 5.1 Execution Results

**✅ Task 3 Successfully Completed**

The Professional Trader agent has successfully generated a comprehensive final trading decision based on the multi-agent investment research from Tasks 1 and 2.

### 5.2 Output Files Generated

**Primary Output:**
- **File Location**: `./results/AMZN_2025-08-19/trader_decision.txt`
- **Content**: Complete trading analysis with final BUY/SELL/HOLD recommendation
- **Format**: Professional trading report with detailed rationale

### 5.3 Key Deliverables

1. **Final Trading Decision**: Clear BUY/SELL/HOLD recommendation
2. **Risk Management Plan**: Position sizing, stop-losses, and profit targets
3. **Execution Strategy**: Entry/exit timing and order management
4. **Market Analysis**: Technical and fundamental factor assessment
5. **Scenario Planning**: Contingency plans for different market outcomes

### 5.4 Professional Standards Met

- ✅ Institutional-grade decision framework implemented
- ✅ Comprehensive risk management protocols established
- ✅ Clear audit trail and decision documentation provided
- ✅ Actionable implementation guidelines delivered
- ✅ Multi-agent research synthesis completed

## 6. Complete 


**🏆 Congratulations! Task 3 has been successfully completed with professional-grade trading analysis and decision-making capabilities.**